# Using a SurrealDB Knowedge Graph and Gemini to build a Graph-RAG 

This Jupyter Notebook illustrates the implementation and utilization of a graph-based Retrieval-Augmented Generation (RAG) system using a knowledge graph stored in SurrealDB. It focuses on leveraging the knowledge graph created by the graph_extractor.ipynb notebook for question answering tasks.

### Key Functionality
####  Data Loading:
        Loads the raw text data from which will be used for context in the prompt.

#### Leveraging the Existing Knowledge Graph:

        Create a SurrQL function to query the knowledge graph using vector search for the descriptions of the entities and the other entities and relationships they have in the graph. This data will also be fed into the prompt for context.

        
#### Response Generation without a knowledge graph:
    Using a template prompt in the RAG_prompts.py file a simple context window of the text data loaded earlier with the question is asked of the LLM model.

    

#### Response Generation with knowledge graph:
    Using a template prompt in the RAG_prompts.py file a context window of the text data loaded earlier, plus the graph data retrieved from SurrealDB with the question is asked of the LLM model.


##### notes:
    This notebook utilizes libraries :
        surrealdb to interact with SurrealDB
        google.generativeai to interact with Gemini
        networkx to visualize the graph

    Prerquisite is to install the embedding model as in this python script:
            https://github.com/apireno/surrealDB_embedding_model
            and load a knowledge graph using the other notebook in this repo: graph_extractor.ipynb

    The notebook is loosely based on a larger repository microsoft/graphrag. The microsoft/graphrag repository provides a more comprehensive, modular graph-based RAG system. This notebook focuses specifically on the knowledge graph construction aspect.

In [10]:
import sys
import os
import json
import google.generativeai as genai
import os
import time
import ipynb_path
import pandas as pd
from surrealdb import AsyncSurrealDB

from IPython.display import display, Markdown,JSON
from IPython.display import JSON


#get this notebook's path for access to the other files needed
dir_path = os.path.dirname(os.path.realpath(ipynb_path.get(__name__)))
sys.path.append(dir_path) #add the current directory for adding py imports
from RAG_prompts import STD_RAG_PROMT, GRAPH_RAG_PROMT

In [2]:

# this folder
nb_folder = dir_path
input_file = nb_folder + "/Operation Dulce v2 1 1.md"

out_folder = nb_folder + "/rag_{0}".format(time.strftime("%Y%m%d-%H%M%S"))
os.makedirs(out_folder, exist_ok=True)
chat_file = out_folder + "/chat.json"

messages_to_log = []



ip = "0.0.0.0:8000"
url = "ws://{0}".format(ip)

u = "root"
p = "root"
n = "graph_rag"
d = "graph_rag"
db_folder = nb_folder + "/db"

surrealdb_start = "surreal start --allow-net --log none --user {u} --pass {p} --bind {ip} \"rocksdb://{db_folder}\"".format(
    u=u,
    p=p,
    ip=ip,
    db_folder=db_folder)

#run this command if your surreal instance isn't running yet 
#copy and paste from below into a terminal
print(surrealdb_start)

#and ensure you installed the embedding model!
#the model will power the function fn::sentence_to_vector($text)
print("""
ensure you installed the embedding model!
      
      https://github.com/apireno/surrealDB_embedding_model

the model will power the function fn::sentence_to_vector($text)
"""   )  

surreal start --allow-net --log none --user root --pass root --bind 0.0.0.0:8000 "rocksdb:///Users/sandro/git_repos/graph_rag/db"

ensure you installed the embedding model!
      
      https://github.com/apireno/surrealDB_embedding_model

the model will power the function fn::sentence_to_vector($text)



In [3]:

# get the text from the source file for use in our prompt
with open(input_file, "r") as f:
    input_text = f.read()


entity_types = ["PERSON", "PLACE"]
relation_type = "RELATED_TO"

#create the function that will return the portion of the knowledge graph 
#relevant to the question using vector search on the description embedding
retrieve_function_surql = """
DEFINE FUNCTION OVERWRITE fn::get_related_entities($text: string) {{
    
    LET $emb = fn::sentence_to_vector($text);
    
    RETURN SELECT id,description, ->{1}.{{out,strength,description}} FROM {0} 
        WHERE  embedding <|5,COSINE|> $emb;
}};
""".format(entity_types[0],relation_type)


async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    outcome = await db.query(retrieve_function_surql)



In [67]:
def transform_record_ids(data):
  """
  Transforms the input data to extract the 'ID' field from 'out' and 'in' objects.

  Args:
    data: An array of dictionaries.

  Returns:
    The transformed array of dictionaries.
  """
    
  for item in data:
    if "id" in item:
        item["id"] = item["id"].id
    if "->RELATED_TO" in item:
      for inner_dict in item["->RELATED_TO"]:
        if "out" in inner_dict and hasattr(inner_dict["out"], "id"):
          inner_dict["out"] = inner_dict["out"].id
        if "in" in inner_dict and hasattr(inner_dict["in"], "id"):
          inner_dict["in"] = inner_dict["in"].id
        if "id" in inner_dict and hasattr(inner_dict["id"], "id"):
          inner_dict["id"] = inner_dict["id"].id
      
    
  return data


In [70]:

#Let's test out vector/graph query
#we should see a collection of people and relationships
#where the person's description is related to the question 

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    get_the_graph_context_surql = "fn::get_related_entities('{0}')"
    outcome = await db.query(get_the_graph_context_surql.format("Who is the main character?"))

result = transform_record_ids(outcome[0]["result"])
#print(result[0])

#df["out"] = df["out"].apply(lambda x: x.id) 
#print(df["->RELATED_TO"].apply(lambda x: pd.DataFrame(x)["out"]))
JSON(result)
    

<IPython.core.display.JSON object>

In [71]:


def generate_content_with_attachment(the_model,the_messages,attached_file):
    #time.sleep(5) # to stop generating too many requests per min

    response = the_model.generate_content([attached_file,str(the_messages)])
    return response





def generate_std_LLM_messages(the_question,attached_file):
    messages =  [
            {"role": "user", "parts": [{"text": STD_RAG_PROMT.format(
                question = the_question
            )}]}
        ]
    return messages


#this function will call the LLM with a prompt that includes the text of the original document
def get_std_LLM_response(the_model,the_question,attached_file):
    
    messages = generate_std_LLM_messages(the_question,attached_file)
    messages_to_log.append(messages)
    
    with open(chat_file, "w") as f:
        json.dump(messages_to_log, f)
        
    response = generate_content_with_attachment(model,messages,attached_file)
    return response

   
async def generate_graph_RAG_messages(the_db,the_question,attached_file):
    get_the_graph_context_surql = "fn::get_related_entities('{0}')"
    
    outcome = await the_db.query(get_the_graph_context_surql.format(the_question))
    messages = [
            {"role": "user", "parts": [{"text": GRAPH_RAG_PROMT.format(
                question = the_question,
                knowledge_graph = outcome
            )}]}
        ]
    
    return messages
    
    
#this function will call the LLM with a prompt that includes the text of the original document
#plus the relevant entities and relationships from the knowledge graph bases on a vector search
async def get_graph_RAG_response(the_db,the_model,the_question,attached_file):
    
    messages = await generate_graph_RAG_messages(the_db,the_question,attached_file)
    messages_to_log.append(messages)
    
    with open(chat_file, "w") as f:
        json.dump(messages_to_log, f)
    
    response = generate_content_with_attachment(model,messages,attached_file)
    return response
    
       

In [72]:

# Set your API key (I've stored mine in an env variable)
genai.configure(api_key=os.getenv('GOOGLE_GENAI_API_KEY'))

#flash is faster and good enough
#model = "gemini-1.0-pro"
model = "gemini-1.5-flash"

model = genai.GenerativeModel(model)


the_attached_file = genai.upload_file(input_file)

while the_attached_file.state.name == "PROCESSING":
    print(".", end="")
    time.sleep(10)
    the_attached_file = genai.get_file(the_attached_file.name)





In [73]:
q = "Who is the main character and what motivates them?"

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")
messages = generate_std_LLM_messages(q,the_attached_file)
print(json.dumps(messages[0], indent=4, sort_keys=True))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    messages = await generate_graph_RAG_messages(db,q,the_attached_file)
    print(json.dumps(messages[0], indent=4, sort_keys=True))
 
   






 ------ STD RAG -------------------------




{
    "parts": [
        {
            "text": "\n-Goal-\nYou are a researcher who is providing answers to a question for you client based on documents provided you. Answer the questions to the best of your ability given the documents.\n\n<question>\nWho is the main character and what motivates them?\n</question>\n\n "
        }
    ],
    "role": "user"
}




 ------ GRAPH RAG -------------------------




{
    "parts": [
        {
            "text": "\n-Goal-\nYou are a researcher who is providing answers to a question for you client based on documents provided you. You also have access to a knowledge graph based on the documents provided. Answer the questions to the best of your ability given the documents and knowledge graph.\n\n<knowledge graph>\n[{'result': [{'->RELATED_TO': [{'description': 'Alex Mercer operates within Dulce Base during Operation: Dulce.', 'out': RecordID(table_name=PLACE, record_id=DULCE BASE), 'strength': 9},

In [75]:
#Lets test a question...

q = "Who is the main character and what motivates them? Format your response in bullet points."

response = get_std_LLM_response(model,q,the_attached_file)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")



display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,the_attached_file)
    display(Markdown(response.text))








 ------ STD RAG -------------------------






* **Main Character:** Agent Alex Mercer is arguably the main character, though the narrative is ensemble-focused, giving significant attention to the perspectives and motivations of other key players.

* **Motivations:**
    * **Duty and Obedience:** Mercer is a highly trained agent, deeply committed to following protocols and completing his assigned mission.  This is demonstrated by his initial compliance with Agent Cruz's directives, though this is gradually challenged throughout the story.
    * **Discovery and Understanding:** Despite his inherent obedience, Mercer is driven by a powerful desire to understand the mysteries of Dulce Base and the alien communication. This intellectual curiosity clashes with his ingrained sense of duty, creating internal conflict that fuels his actions.
    * **Protection of Humanity:** Mercer's concern for the safety and well-being of humanity is a significant motivator. His ultimate goal is to ensure that first contact with extraterrestrial intelligence is handled responsibly and doesn't lead to harm.
    * **Teamwork and Collaboration:** As the story progresses, Mercer demonstrates a growing reliance on and trust in his team members. His leadership style shifts from strict adherence to protocol to collaborative problem-solving, emphasizing the importance of combined human insight to navigate the unprecedented challenges they face.

While other characters like Dr. Jordan Hayes (driven by scientific curiosity) and Agent Taylor Cruz (motivated by ambition and control) also play pivotal roles, Mercer's character arc, spanning from strict compliance to a more nuanced understanding of the mission, positions him as the central figure through which the overarching narrative unfolds.






 ------ GRAPH RAG -------------------------






* **Main Character:** Agent Alex Mercer is the main character of the story.  While the narrative features other important agents, Alex's perspective is central, and his actions and internal conflicts drive much of the plot.

* **Motivations:** Alex is motivated by several factors:

    * **Duty and Responsibility:** As a high-ranking member of the Paranormal Military Squad, he feels a strong sense of duty to complete his mission and protect national security. This is evident in his compliance with Agent Cruz’s orders and his concern for the safety of his team.
    * **Determination and Resolve:**  Alex possesses unwavering determination and displays resilience in the face of the unknown. He demonstrates this through his consistent engagement with the mission even when uncertain and in the face of challenges.
    * **A Hunger for Knowledge:** Alex shows significant curiosity and a desire for understanding beyond following orders. This is seen in his conversations with Dr. Hayes, where he acknowledges a sense of being pulled into "someone else's game" and seeks deeper truths beyond the official narrative. He is driven to discover and understand the mysteries of Dulce Base.
    * **Mentorship:** Alex acts as a mentor to Sam Rivera, guiding and supporting their contributions. This suggests a desire to foster growth and share his expertise, indicating a sense of responsibility beyond his own mission.
    * **Survival:** While duty and knowledge drive him, survival and the protection of his team are also strong motivating factors. This is implicit in his actions throughout the story as he continuously assesses risks and strives for the team's success.



In [76]:

#Let's test a question that asks about the main character's relationships
q = "What places and people does Alex Mercer hold dearest? Format your response in bullet points. Format your response in bullet points."

response = get_std_LLM_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")


display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))







 ------ STD RAG -------------------------






Based solely on the provided text, there is no explicit mention of places or people Alex Mercer holds dearest.  The text focuses on his professional life and his participation in Operation: Dulce.  While he shows camaraderie and respect for his team members (Jordan Hayes, Taylor Cruz, and Sam Rivera),  there's no indication of these relationships extending beyond professional respect and shared mission objectives.  The narrative doesn't offer details about his personal life or attachments outside of his work with Paranormal Military Squad.






 ------ GRAPH RAG -------------------------






Based on the provided text and knowledge graph, here's what we can infer about the places and people Alex Mercer holds dearest:

* **Places:**  The knowledge graph strongly indicates Dulce Base as a significant location for Alex Mercer due to his involvement in Operation: Dulce.  While the text doesn't explicitly state he holds it *dear*, its importance to his mission suggests a strong connection. Washington D.C. is also mentioned, representing his chain of command and source of orders.  Whether he holds Washington dear is unclear, as the connection is purely professional.

* **People:** The knowledge graph highlights strong relationships with Jordan Hayes and Sam Rivera, indicating a close working relationship and mutual respect. The strength of these connections (9 and 7 respectively) suggests a level of camaraderie and trust exceeding mere professional obligation. Taylor Cruz is also mentioned, but with a lower strength rating (8), indicating a working relationship with less personal closeness.  There is no information in the provided text to suggest he holds any of these individuals "dearest" in a personal, emotional sense, rather that these are people he works closely with, trusts, and respects professionally.  The text suggests he respects Jordan's intellect highly.


In [78]:


q = """


Create a table of places that Alex communicates with:


In the first column indicate the person. 
In the 2nd column list the frequency he has scenes with them.
In the 3rd column list the places they had scenes together ordered by importance of the place and scene.
In the 4th column indicate the strength of the bond between the people.
In the 5th column score the total weight of the interactions across the scenes in aggregate.

output the response in a table format using markdown language.
"""


response = get_std_LLM_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")

display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))





 ------ STD RAG -------------------------






## Alex Mercer's Communication Network

| Person             | Communication Frequency | Places of Communication (Ordered by Importance)                                                                   | Bond Strength | Interaction Weight Score |
|----------------------|--------------------------|-----------------------------------------------------------------------------------------------------------------|-----------------|--------------------------|
| Jordan Hayes         | Very High                 | 1. Dulce Base Command Center (multiple scenes, critical collaborations), 2. Dulce Base Corridors (shared concerns), 3. Elevator to Dulce (shared apprehension), 4. Dulce Base Lab (scientific collaboration) | Very Strong     | 9                        |
| Taylor Cruz          | High                      | 1. Dulce Base Command Center (conflicts and uneasy alliances), 2. Briefing Room (initial instructions, later collaboration), 3. Equipment Bay (philosophical disagreement) | Strong but strained | 7                        |
| Sam Rivera           | High                      | 1. Dulce Base Command Center (collaborative work, shared excitement), 2. Sam's Office (advice and encouragement), 3. Dulce Base Corridors (shared mission) | Strong           | 7                        |


**Note:**  The "Interaction Weight Score" is a subjective assessment based on the length and significance of their interactions throughout the provided text.  A higher score indicates more frequent and/or more impactful interactions.  The ordering of places within each person's row reflects the apparent importance of the setting to their relationship and the overall narrative.






 ------ GRAPH RAG -------------------------






Based on the provided text and knowledge graph, here's a table summarizing Alex Mercer's communication and interactions,  keeping in mind that the provided knowledge graph lacks information on the *frequency* of scenes and requires inference from the narrative.  The "Strength of Bond" and "Total Interaction Weight" columns are estimations based on the narrative and the knowledge graph's strength scores.  Precise quantification isn't possible without more structured data.

| Person           | Frequency of Scenes | Places of Interaction (ordered by importance)                                    | Strength of Bond | Total Interaction Weight (estimated) |
|-------------------|----------------------|---------------------------------------------------------------------------------|--------------------|------------------------------------|
| Jordan Hayes      | Very High             | Dulce Base Command Center, Mainframe Room, Communications Hub, Underground Meeting Room | 9                  | 90                                 |
| Sam Rivera        | High                  | Dulce Base Command Center, Mainframe Room, Server Room, Underground Meeting Room      | 8                  | 72                                 |
| Taylor Cruz       | High                  | Dulce Base Command Center, Mainframe Room, Underground Meeting Room, Briefing Room | 7                  | 63                                 |


**Notes:**

* **Frequency:**  This column is an approximation based on the narrative's emphasis on the interactions between characters.  "Very High" indicates frequent, significant interactions throughout the story. "High" suggests numerous important interactions.
* **Places:** The order of places reflects the narrative's emphasis on the importance of each location within the context of their interactions.  For instance, the Dulce Base Command Center is listed first because many major interactions occur there.
* **Strength of Bond:** This is a subjective assessment based on the descriptive language in the text and the strength scores from the knowledge graph.  It reflects the level of trust, cooperation, and understanding between characters.
* **Total Interaction Weight:** This is a rough estimate, multiplying the "Frequency" (interpreted as a numerical value: Very High=10, High=8) by the "Strength of Bond". It attempts to quantify the overall significance of the interactions.


The knowledge graph is limited. A more comprehensive graph that includes scene frequency and interaction details would allow for a more precise and detailed response.


In [77]:


q = """

Create a table of places that Alex visits:


In the first column indicate the place name. 
In the 2nd column indicate the frequency he visits them.
In the 3rd column list the people who were there with him by strength of the bond of the people.
In the 4th column indicate the importance of the place.
In the 5th column describe the location in 255 characters or less.
In the 6th column score the total weight of the interactions across the scenes in aggregate.

output the response in a table format using markdown language.
"""



response = get_std_LLM_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")

display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))





 ------ STD RAG -------------------------






| Place Name             | Frequency | People Present (Bond Strength)                     | Importance                               | Location Description                                                                     | Interaction Weight |
|-------------------------|------------|----------------------------------------------------|-------------------------------------------|-----------------------------------------------------------------------------------------|--------------------|
| Briefing Room           | 3          | Cruz (Antagonistic), Hayes (Understanding), Rivera (Respectful),  | High (Mission planning, strategy)      | Sterile, large room with projectors; used for mission briefings.                         | 7                 |
| Cramped Underground Room| 1          | Hayes, Cruz                                         | High (Mission initiation, descent)       | Small, dark room, leading to elevator to Dulce.                                        | 3                 |
| Elevator Shaft          | 1          | Hayes, Cruz, Rivera                                | High (Journey to Dulce)                  | Metal shaft, descending into Dulce base.  Dark and claustrophobic.                     | 3                 |
| Dulce Base Corridors    | 3          | Hayes (Strong), Cruz (Antagonistic), Rivera (Respectful)     | High (Exploration, discoveries)          | Long, dark corridors within Dulce base. Steel doors, eerie atmosphere.                | 8                 |
| Dulce Base Mainframe Room| 2          | Hayes (Strong), Cruz (Antagonistic), Rivera (Respectful)     | High (Central hub, communication)      | Large room containing mainframe and communications equipment.                             | 7                 |
| Dulce Base Lab          | 1          | Hayes (Strong)                                       | Moderate (Alien tech analysis)           | A laboratory within Dulce base; contains alien technology.                            | 2                 |
| Dulce Base Server Room   | 1          | Rivera (Mentor-Mentee),                                  | Moderate (Data analysis)                 | Small room with numerous servers; used for data analysis.                               | 2                 |
| Dulce Base Command Center| 6          | Hayes (Strong), Cruz (Antagonistic), Rivera (Respectful) | Highest (Alien communication, mission culmination)| Central hub of Dulce base, containing advanced communications equipment and consoles. | 15                |


**Note:** Bond strength is a subjective assessment based on the narrative. Interaction weight is an aggregate score reflecting the significance of interactions in each location across all chapters, with higher scores indicating greater importance and interaction.  Frequency counts instances where Alex is explicitly present and actively participating.






 ------ GRAPH RAG -------------------------






Based on the provided text and knowledge graph, here's a table summarizing the places Alex Mercer visits, along with associated details:

| Place Name        | Visit Frequency | People Present (by strength of bond) | Importance of Place | Location Description                                                                        | Total Interaction Weight |
|--------------------|-----------------|--------------------------------------|----------------------|--------------------------------------------------------------------------------------------|-------------------------|
| Dulce Base         | Multiple        | Jordan Hayes (9), Taylor Cruz (8), Sam Rivera (7) | Very High             | Underground military base; mysterious, potentially dangerous.                              | 24                       |
| Briefing Room      | Multiple        | Jordan Hayes, Taylor Cruz, Sam Rivera | High                   | Sterile room where missions are briefed.                                                    | 24                       |
| Cramped Office     | At least once   | Sam Rivera                         | Medium                 | Small office where Sam works with servers; cramped workspace.                             | 7                        |
| Deserted Corridor  | At least once   | Taylor Cruz                         | Low                    | A hallway inside Dulce Base.                                                               | 6                        |
| Secluded Equipment Bay | At least once | Taylor Cruz, Jordan Hayes             | Low                    | Equipment storage area inside Dulce Base.                                                   | 11                       |
| Underground Meeting Room | At least once | Jordan Hayes, Taylor Cruz, Sam Rivera | Medium                 | Small underground room near elevator to Dulce Base.                                        | 24                       |
| Cramped Underground Meeting Room | At least once | Jordan Hayes, Taylor Cruz, Sam Rivera | Medium                 | Small underground room near elevator to Dulce Base.                                        | 24                       |
| Central Communications Hub | At least once | Jordan Hayes, Taylor Cruz, Sam Rivera | High                   | Main communications area within Dulce Base.                                              | 24                       |
| Central Hub        | At least once   | Jordan Hayes, Taylor Cruz, Sam Rivera | High                   | Main access point to deeper areas within Dulce Base.                                  | 24                       |
| Mainframe Room      | At least once   | Jordan Hayes, Taylor Cruz, Sam Rivera | High                   | Dulce Base's mainframe room; contains important computers and servers.                      | 24                       |
| Lab                 | At least once   | Jordan Hayes, Taylor Cruz            | High                  | Area within Dulce Base where alien technology is studied.                                | 15                       |
| Server Room         | At least once   | Sam Rivera, Alex Mercer             | Medium                 | Room containing multiple servers.                                                           | 7                        |
| Washington D.C.    | Multiple        | Multiple (unspecified)              | High                    | Government headquarters; source of orders and information.                               |  Unknown                 |


**Note:**  The "Total Interaction Weight" is a rough estimate based on the strength of relationships mentioned in the knowledge graph and the frequency of interactions described in the text.  The Washington D.C. entry has an unknown weight because the specific people Alex interacted with there aren't explicitly stated.  The frequency of visits to some locations is inferred from the narrative.


In [78]:



q = """
    Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Describe this cover in 20 words or less.
"""

response = get_std_LLM_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")

display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p)  
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))






 ------ STD RAG -------------------------






Alex and Jordan, bathed in alien light, decipher a cosmic message in Dulce Base's command center.  Hope and fear intertwine in their shared gaze.






 ------ GRAPH RAG -------------------------






Here's a book cover description based on your request:

**Image:** Alex Mercer and Jordan Hayes, silhouetted against Dulce Base's glowing entrance, hand reaching towards an alien signal.

**Text:**  *Dulce:  A cosmic encounter in the heart of darkness.*


In [79]:


q = """Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Draw this cover in the syle of an 8-bit video game.
"""

response = get_std_LLM_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")

display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))





 ------ STD RAG -------------------------






## Operation: Dulce - Book Cover Concept

**Scene Choice:** The most emotionally charged and important scene for the cover, considering Alex Mercer's strongest relationships and plot significance, is the climax of Chapter 11 or a similar scene from later chapters where the team is actively engaging in interstellar communication for the first time.  This captures the core of the narrative:  the risky first contact with an alien intelligence, highlighting the collaborative effort of the team.

**8-Bit Style Cover Art Description:**

The cover art will be rendered in a classic 8-bit video game style, reminiscent of NES or early SNES games.  The overall color palette will be dark, using blues, purples, and greens to create a mysterious and slightly ominous atmosphere.  

**Foreground:**

* **Alex Mercer:**  A pixelated Alex Mercer will be centrally positioned, but slightly off-center, his expression a mix of determination and apprehension.  He is depicted in his military uniform, but maybe with a futuristic twist; his weapon is slightly visible, hinting at the ongoing tension. He's looking towards a central point.

* **Communication Interface:** The foreground will prominently feature a stylized, 8-bit representation of a complex communication interface, similar to the monitors described in the text.   Pixelated symbols and numbers (both human and alien) will scroll across the screen. The alien symbols will be distinct and evocative of a higher, or different, order of intelligence.  Bright neon colors (blue and green) will highlight active parts of the interface.

**Background:**

* **The Dulce Base:**  A simplified representation of the Dulce Base will be in the background. It can be suggested through pixelated grey and brown blocks representing underground structures. Maybe the elevator shaft and the corridors are visible. The style should imply depth and mystery.

* **Cosmic Elements:**  Subtle hints of cosmic imagery will subtly appear in the background; perhaps a few larger pixels representing distant stars or nebulas. This background element won't overpower the foreground but adds a sense of scale and the wider implications of the operation.

* **Team Silhouettes:**  The silhouettes of Jordan, Taylor, and Sam will be partially visible behind Alex and the interface.  This underscores the collaborative nature of the mission and their crucial roles.


**Overall Style:**

The style will be intentionally retro, using a limited color palette and simple pixel art to evoke a sense of nostalgia while remaining visually captivating.  The overall feeling should be one of mystery, suspense, and high-stakes action, promising an engaging and thought-provoking story. The title, "Operation: Dulce," will be rendered in a bold, pixelated font, matching the retro aesthetic.


**Specific Pixel Art Elements:**

* Use a pixel art style that reflects the technology of the 80s and 90s.
* The characters should have distinct features and expressions even with a limited number of pixels.
*  Use vibrant colors sparingly to draw the eye to specific details, like the communication interface.  The rest will be in muted shades.
* Incorporate shading through pixel manipulation to give characters and the background depth.
* Use the limited palette and pixel style to convey emotions such as tension, anticipation, and determination.


This concept aims to create a visually compelling book cover that immediately captures the essence of the story—a suspenseful, science fiction adventure centered around a crucial first contact event. The retro 8-bit style adds a layer of unique character and intrigue, promising a story with equal parts scientific exploration and human drama.






 ------ GRAPH RAG -------------------------






Here's a description of a book cover scene and its depiction in an 8-bit video game style, based on your provided text and knowledge graph:


**Cover Scene Choice:**

The most emotionally charged and important scene, considering Alex Mercer's strongest relationships (Jordan Hayes and Sam Rivera) and key locations (Dulce Base), is the moment when the team first receives a clear response from the alien signal.  This moment represents a pivotal shift from simple investigation to actual interstellar communication, and the emotional weight of that realization is a key theme of the story. The scene where Sam Rivera is shown to be the key in obtaining the communication is particularly important, representing his evolution in the story and his pivotal role.


**8-bit Video Game Style Cover Art Description:**

The cover art would be a pixelated scene set primarily within the Dulce Base command center.

* **Background:** The background would consist of large, chunky pixels representing the dimly lit control room.  Shades of dark gray and blue would dominate, with some brighter pixels to represent glowing consoles and monitors.  The walls could feature pixelated depictions of pipes and vents, subtly conveying the underground setting.

* **Characters:**
    * **Alex Mercer:**  A sprite with a determined expression, depicted in shades of green and brown, would be centrally positioned, partially obscuring the monitor's display. His sprite would have some pixelated details to indicate his uniform and equipment.
    * **Jordan Hayes:**  A sprite slightly smaller than Alex, maybe in purple and light blue, would be positioned next to Alex, their hand hovering over a pixelated keyboard.
    * **Sam Rivera:** A smaller sprite in bright orange or yellow would be positioned slightly below Alex and Jordan, intensely focused on their own console, representing their integral role in the moment.
    * **Taylor Cruz:** A sprite in dark blue/gray (to represent their authoritative nature) would be partially visible in the background, watching the others.

* **Foreground:** The foreground would feature a brightly colored, pixelated representation of the alien signal itself.  It could be a series of geometric shapes, vibrant colors, or even stylized alien symbols, to capture the extraordinary nature of the communication.  The signal should be partially overlapping the foreground to suggest its integration within the scene.

* **Text:**  The title, "Operation: Dulce," would appear in a bold, pixelated 8-bit font at the top of the cover. The author's name would be smaller, below the title.  A subtitle like "First Contact" or "The Interstellar Symphony" could be added below the title.


**Overall Style:**

The style should evoke classic 8-bit RPGs or adventure games.  The color palette should be limited, using bold colors for contrast and impact.  The pixel art should be relatively high-resolution compared to truly vintage 8-bit graphics to keep it visually appealing, yet still distinctly 8-bit. The overall impression should be one of mystery, tension, and excitement, reflecting the narrative of the book.


In [80]:


imagen = genai.ImageGenerationModel("imagen-3.0-generate-001")

result = imagen.generate_images(
    prompt="Fuzzy bunnies in my kitchen",
    number_of_images=4,
    safety_filter_level="block_only_high",
    person_generation="allow_adult",
    aspect_ratio="3:4",
    negative_prompt="Outside",
)

for image in result.images:
  print(image)

# Open and display the image using your local operating system.
for image in result.images:
  image._pil_image.show()





NotFound: 404 models/imagen-3.0-generate-001 is not found for API version v1beta, or is not supported for predict. Call ListModels to see the list of available models and their supported methods.

In [81]:

q = """Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Draw this cover in the syle of an 8-bit video game using imagen3.
"""

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")
messages = generate_std_LLM_messages(q,the_attached_file)
print(messages[0]["parts"][0]["text"])

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    messages = await generate_graph_RAG_messages(db,q,the_attached_file)
    print(messages[0]["parts"][0]["text"])








 ------ STD RAG -------------------------





-Goal-
You are a researcher who is providing answers to a question for you client based on documents provided you. Answer the questions to the best of your ability given the documents.

<question>
Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Draw this cover in the syle of an 8-bit video game using imagen3.

</question>

 




 ------ GRAPH RAG -------------------------





-Goal-
You are a researcher who is providing answers to a question for you client based on documents provided you. You also have access to a knowledge graph based on the documents provided. Answer the questions to the best of your ability given the documents and knowledge graph.

<knowledge graph>
[{'result': [{'->RELATED_TO': [{'description': 'Alex Mercer operates within Dulce Base during Operation: Dulce.', 'out': Recor